In [1]:
import numpy as np

In [5]:
def custom_fft(x):
    return np.fft.fft(x)

In [6]:
def custom_ifft(X):
    return np.fft.ifft(X)


In [7]:
def custom_overlap_add(signal, impulse_response, block_size):
    signal_length = len(signal)
    impulse_length = len(impulse_response)
    impulse_padded = np.concatenate((impulse_response, np.zeros(block_size - impulse_length)))
    H = custom_fft(impulse_padded)
    output_signal = np.zeros(signal_length + impulse_length - 1)
    for i in range(0, signal_length, block_size - impulse_length + 1):
        chunk = signal[i:i + block_size - impulse_length + 1]
        chunk = np.concatenate((chunk, np.zeros(impulse_length - 1)))
        X = custom_fft(chunk)
        Y = X * H
        y = custom_ifft(Y)
        output_signal[i:i + block_size - impulse_length + 1 + impulse_length - 1] += np.real(y)
    return output_signal[:signal_length + impulse_length - 1]

In [8]:
def custom_overlap_save(signal, impulse_response, block_size):
    signal_length = len(signal)
    impulse_length = len(impulse_response)
    if block_size < signal_length + impulse_length - 1:
        block_size = signal_length + impulse_length - 1
    impulse_padded = np.concatenate((impulse_response, np.zeros(block_size - impulse_length)))
    H = custom_fft(impulse_padded)
    signal = np.concatenate((np.zeros(impulse_length - 1), signal))
    output_signal = np.zeros(signal_length)
    for i in range(0, signal_length, block_size - impulse_length + 1):
        chunk = signal[i:i + block_size]
        chunk_padded = np.concatenate((chunk, np.zeros(block_size - len(chunk))))
        X = custom_fft(chunk_padded)
        Y = X * H
        y = custom_ifft(Y)
        output_signal[i:i + block_size - impulse_length + 1] += np.real(y[impulse_length - 1:impulse_length + block_size - impulse_length])
    return output_signal[:signal_length + impulse_length - 1]

In [9]:
x_n = np.array([-1,2,-5,6,7,3,-2,-4,9,1,-3,8]) 
h_n = np.array([1,1,0]) 
N = 8  
L = 6  
M = len(h_n) 

In [10]:
y_n = custom_overlap_add(x_n, h_n, N)
print(y_n)

[-1.  1. -3.  1. 13. 10.  1. -6.  5. 10. -2.  5.  8.  0.]


In [11]:
y_n = custom_overlap_save(x_n, h_n, N) #y[n]
print(y_n)

[-1.  1. -3.  1. 13. 10.  1. -6.  5. 10. -2.  5.]
